- Model1 Actual: gs://iorek-byrnison/tc-cicd-dir/out/trained_model_20220317_071935
- Model2 Synthetic: gs://iorek-byrnison/tc-cicd-dir/out/trained_model_20220317_071952

In [16]:
!gsutil cp gs://iorek-byrnison/tc-cicd-dir/data/tc-data/test.csv .

Copying gs://iorek-byrnison/tc-cicd-dir/data/tc-data/test.csv...
/ [1 files][183.9 KiB/183.9 KiB]                                                
Operation completed over 1 objects/183.9 KiB.                                    


In [18]:
!head test.csv

0967-BMLBD,Female,0,Yes,Yes,11,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,20.0,196.35,No
2361-FJWNO,Male,0,No,No,40,No,No phone service,DSL,No,Yes,No,Yes,No,No,One year,No,Credit card (automatic),36.0,1382.9,No
9830-ECLEN,Male,0,Yes,Yes,58,No,No phone service,DSL,Yes,Yes,Yes,No,Yes,No,One year,No,Mailed check,50.0,2919.85,No
3275-RHRNE,Male,0,Yes,Yes,28,No,No phone service,DSL,No,Yes,Yes,Yes,Yes,Yes,One year,Yes,Credit card (automatic),60.0,1682.05,No
1183-CANVH,Female,0,Yes,No,23,Yes,No,DSL,No,No,No,Yes,Yes,No,One year,No,Bank transfer (automatic),60.0,1347.15,No
1088-CNNKB,Male,0,Yes,No,63,Yes,Yes,DSL,Yes,Yes,Yes,Yes,No,Yes,One year,No,Credit card (automatic),80.0,5040.2,No
0727-BMPLR,Female,1,No,No,55,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,One year,Yes,Electronic check,100.0,5509.3,Yes
0512-FLFDW,Female,1,Yes,No,60,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,One year,Yes,C

In [28]:
import os, sys
import tensorflow as tf
sys.path.append(os.path.abspath('../tc_trainer/'))

In [48]:
from trainer.data import load_dataset
testds = load_dataset('test.csv', 2000, num_repeat=1)

### Base Line Model (actual dataset) against test data

In [132]:
model_path = 'gs://iorek-byrnison/tc-cicd-dir/out/trained_model_20220317_071935/savedmodel'
base_line_model1 = tf.keras.models.load_model(model_path)
base_line_model1.evaluate(testds)

1/1 [==============================] - 1s 1s/step - loss: 0.6032 - accuracy: 0.6902


[0.6032319664955139, 0.6902016997337341]

### Base Line Model (synthetic dataset) against test data

In [133]:
model_path = 'gs://iorek-byrnison/tc-cicd-dir/out/trained_model_20220317_071952/savedmodel'
base_line_model2 = tf.keras.models.load_model(model_path)
base_line_model2.evaluate(testds)

1/1 [==============================] - 1s 1s/step - loss: 0.6932 - accuracy: 0.7673


[0.6931938529014587, 0.7672910690307617]

### Base Line Model (synthetic dataset) against test data - custom containers

In [134]:
model_path = 'gs://iorek-byrnison/tc-cicd-dir/out/trained_model_20220317_081921/savedmodel'
base_line_model3 = tf.keras.models.load_model(model_path)
base_line_model3.evaluate(testds)

1/1 [==============================] - 1s 1s/step - loss: 0.6531 - accuracy: 0.7615


[0.6531007289886475, 0.7615273594856262]

### HPT Model (synthetic dataset) against test data

In [135]:
model_path = 'gs://iorek-byrnison/tc-cicd-dir/out/tc_churn_hpt_model1/6/model/savedmodel'
base_line_model4 = tf.keras.models.load_model(model_path)
base_line_model4.evaluate(testds)

1/1 [==============================] - 3s 3s/step - loss: 0.8182 - accuracy: 0.7788


[0.8182476758956909, 0.7788184285163879]

#KFP Lightweight

#tc-kfp-lightweight-pipeline-20220317083635

- best_accuracy, 0.8048916813470339
- best_alpha, 0.022156066144598474
- best_max_iter, 1

In [57]:
#Prebuilt: tc-kfp-prebuilt-create-endpoint-job
#Lightweight: churn_kfp_model_endpoint

In [72]:
from google.cloud import aiplatform
ep_list = aiplatform.Endpoint.list()

lightweight_ep = None
prebuilt_ep = None
for ep in ep_list:
    if ep.resource_name == 'projects/1094499562477/locations/us-central1/endpoints/967895687884701696':
        lightweight_ep = ep
        print(f'YES: {lightweight_ep.display_name}')
    
    if ep.resource_name == 'projects/1094499562477/locations/us-central1/endpoints/2724299542559195136':
        prebuilt_ep = ep
        print(f'YES: {prebuilt_ep.display_name}')

YES: tc-kfp-prebuilt-create-endpoint-job
YES: churn_kfp_model_endpoint


In [ ]:
numeric_feature_indexes = slice(0, 10)
categorical_feature_indexes = slice(10, 12)

In [124]:
!gsutil cp gs://iorek-byrnison/tc-cicd-dir/artfact-store/kfp-prebuilt/20220317090015/model/model.pkl .

E0317 10:05:41.269233713   21799 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Copying gs://iorek-byrnison/tc-cicd-dir/artfact-store/kfp-prebuilt/20220317090015/model/model.pkl...
/ [1 files][  4.8 KiB/  4.8 KiB]                                                
Operation completed over 1 objects/4.8 KiB.                                      


In [123]:
import pickle
with open('gs://iorek-byrnison/tc-cicd-dir/artfact-store/kfp-prebuilt/20220317090015/model/model.pkl') as f:
    pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'gs://iorek-byrnison/tc-cicd-dir/artfact-store/kfp-prebuilt/20220317090015/model/model.pkl'

In [116]:
import pandas as pd
from trainer.constants import CSV_COLUMNS
df_test = pd.read_csv('gs://iorek-byrnison/tc-cicd-dir/data/tc-data/test.csv', header=None, names= CSV_COLUMNS)
dict_test = df_test.to_dict(orient='records')


In [120]:
instance = dict_test[0]


In [ ]:
lightweight_ep.predict([list(instance.values())])

In [ ]:
[('customerID', 'key'),
     ('gender', 'cat'),
     ('SeniorCitizen', 'cat'),
     ('Partner', 'cat'),
     ('Dependents', 'cat'),
     ('tenure', 'num'),
     ('PhoneService', 'cat'),
     ('MultipleLines', 'cat'),
     ('InternetService', 'cat'),
     ('OnlineSecurity', 'cat'),
     ('OnlineBackup', 'cat'),
     ('DeviceProtection', 'cat'),
     ('TechSupport', 'cat'),
     ('StreamingTV', 'cat'),
     ('StreamingMovies', 'cat'),     
     ('Contract', 'cat'),
     ('PaperlessBilling', 'cat'),
     ('PaymentMethod', 'cat'),
     ('MonthlyCharges', 'num'),
     ('TotalCharges', 'num'),
     ('Churn', 'label')]

In [85]:

for instance in dict_test:
    endpoint.predict([instance])

{'customerID': '0967-BMLBD',
 'gender': 'Female',
 'SeniorCitizen': 0,
 'Partner': 'Yes',
 'Dependents': 'Yes',
 'tenure': 11,
 'PhoneService': 'Yes',
 'MultipleLines': 'No',
 'InternetService': 'No',
 'OnlineSecurity': 'No internet service',
 'OnlineBackup': 'No internet service',
 'DeviceProtection': 'No internet service',
 'TechSupport': 'No internet service',
 'StreamingTV': 'No internet service',
 'StreamingMovies': 'No internet service',
 'Contract': 'One year',
 'PaperlessBilling': 'Yes',
 'PaymentMethod': 'Mailed check',
 'MonthlyCharges': 20.0,
 'TotalCharges': 196.35,
 'Churn': 'No'}